In [135]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-shl/train (1).csv
/kaggle/input/test-shl/test.csv
/kaggle/input/all-prompts/all_prompts.csv


preprocess
word to vec
lstm
train
predict

In [136]:
#pd.set_option("display.max_colwidth", None)

In [137]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from gensim.models import Word2Vec
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config


In [138]:
d = pd.read_csv("/kaggle/input/train-shl/train (1).csv")
print(d.head())
t = pd.read_csv("/kaggle/input/test-shl/test.csv")
print(t.head())
p = pd.read_csv("/kaggle/input/all-prompts/all_prompts.csv")
print(p)

   Unnamed: 0  promptId uniqueId  \
0           0         1    1_323   
1           1         1    1_238   
2           2         1    1_212   
3           3         1    1_117   
4           4         1    1_229   

                                               essay  evaluator_rating  
0  At present age, our education system is not go...               3.0  
1  I am agree the tightly defined curriculum of o...               4.0  
2  I strongly agree with the statement that tight...               2.0  
3  Our education system is nice quitely but i dis...               2.0  
4  i am totally agree with the statement that tig...               3.0  
   Unnamed: 0  promptId uniqueId  \
0           0         1    1_315   
1           1         1    1_214   
2           2         1    1_196   
3           3         1    1_178   
4           4         1    1_201   

                                               essay  
0  Curriculum has been adopted in many schools. T...  
1  I strongly agre

In [139]:
from sklearn.utils import shuffle
#shuffling data
d = shuffle(d)

In [140]:
#filling null values with space string
d['essay'] = d['essay'].fillna('').apply(str)

In [141]:
d.isnull().sum()

Unnamed: 0          0
promptId            0
uniqueId            0
essay               0
evaluator_rating    0
dtype: int64

In [142]:
d[d["evaluator_rating"] == 0]

,Unnamed: 0,promptId,uniqueId,essay,evaluator_rating
416,416,2,2_223,population is one of the majour problem of all...,0.0
86,86,1,1_103,The curriculum of our education system is good...,0.0
1182,1182,5,5_353,From having observed human beings for about 23...,0.0
144,144,1,1_305,curriculum of our education systemour educatio...,0.0
1158,1158,5,5_164,ERA OF NUCLEAR DEVELOPMENT In this modern worl...,0.0
...,...,...,...,...,...
1151,1151,5,5_2,"As Earth as whole cannot be possible to live ,...",0.0
784,784,3,3_316,"In my opinion, yes technology has become a new...",0.0
444,444,2,2_109,The increasing gap btween rich and poor is pla...,0.0
1142,1142,5,5_21,now a days most of the countries make use of n...,0.0


In [143]:
#splitting data into train and validation set
d1 = d[:900]
d2 = d[900:]
x_train = d1.essay
x_val = d2.essay
# keeping promptID to add it later into matrices
prompt_train = d1.promptId
prompt_val = d2.promptId
y_train = d1.evaluator_rating
y_val = d2.evaluator_rating
x_test = t.essay
prompt_test = t.promptId


In [144]:
type(x_train)

pandas.core.series.Series

In [145]:
import re
stop_words = set(stopwords.words('english'))
def preprocess_sentence(w):
    #changing all character to lowercase
    
    w = w.lower().strip()
    w = re.sub(r"([?.!,¿])", r" \1 ", (w))
    w = re.sub(r'[" "]+', " ", w)
   
    #removing extra characters
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = w.rstrip().strip()
    
    #tokenizing data
    word_tokens = word_tokenize(w)
    filter_sent = []
    
    #removing stopwords
    for word in word_tokens:
        if word not in stop_words:
            filter_sent.append(word)
    return filter_sent


#preproessing essay
x_train = x_train.apply(preprocess_sentence)
x_val = x_val.apply(preprocess_sentence)
x_test = x_test.apply(preprocess_sentence)

In [146]:
#detokenizing data for tfidf
def detokenize(data):
    detoken = []
    for i in data:
        i = ' '.join([str(elem) for elem in i])
        detoken.append(i)
    return detoken

x_train = detokenize(x_train)
x_val = detokenize(x_val)
x_test = detokenize(x_test)


In [147]:
#changing training data into list format
#import library
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(binary = True)

#changing words into vectors, fit training data in tfidfvectorizer
tf = tfidf.fit(x_train)
x_train = tf.transform(x_train)

# transforming validation and test data
x_val = tf.transform(x_val)
x_test = tf.transform(x_test)

In [148]:
x_train.shape

(900, 14918)

In [149]:
# we are getting large feature vector matrices, reduce it using truncateSVD
from sklearn.decomposition import TruncatedSVD
clf = TruncatedSVD(300)
xpca = clf.fit_transform(x_train)

In [150]:
xpca_val = clf.transform(x_val)
xpca_test = clf.transform(x_test)

In [151]:
# adding prompt IDs to train, validation and test data
np.append(xpca,prompt_train)
np.append(xpca_val,prompt_val)
np.append(xpca_test,prompt_test)

array([ 0.23220767, -0.09541594, -0.04812243, ...,  5.        ,
        5.        ,  5.        ])

In [152]:
#reshaping data so that it can fit  into lstm model
xpca = np.reshape(xpca, (xpca.shape[0], 1, xpca.shape[1]))
xpca_val = np.reshape(xpca_val, (xpca_val.shape[0], 1, xpca_val.shape[1]))
xpca_test = np.reshape(xpca_test, (xpca_test.shape[0], 1, xpca_test.shape[1]))

In [153]:
#LSTM model
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model
lstm = get_model()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 1, 300)            721200    
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________


In [154]:
#training model
lstm.fit(xpca, y_train, validation_data = (xpca_val,y_val),batch_size=64, epochs=150)

Epoch 1/150
15/15 [==============================] - 5s 55ms/step - loss: 8.3189 - mae: 2.6956 - val_loss: 6.3748 - val_mae: 2.3168
Epoch 2/150
15/15 [==============================] - 0s 19ms/step - loss: 6.1630 - mae: 2.3186 - val_loss: 3.4720 - val_mae: 1.7175
Epoch 3/150
15/15 [==============================] - 0s 19ms/step - loss: 2.9081 - mae: 1.5439 - val_loss: 1.4160 - val_mae: 1.0271
Epoch 4/150
15/15 [==============================] - 0s 19ms/step - loss: 1.3263 - mae: 0.9438 - val_loss: 1.2203 - val_mae: 0.7921
Epoch 5/150
15/15 [==============================] - 0s 19ms/step - loss: 1.1427 - mae: 0.8314 - val_loss: 1.1599 - val_mae: 0.8178
Epoch 6/150
15/15 [==============================] - 0s 20ms/step - loss: 1.0310 - mae: 0.7929 - val_loss: 1.2140 - val_mae: 0.7740
Epoch 7/150
15/15 [==============================] - 0s 20ms/step - loss: 1.0137 - mae: 0.7843 - val_loss: 1.1469 - val_mae: 0.8418
Epoch 8/150
15/15 [==============================] - 0s 19ms/step - loss: 1.

In [155]:
#predicting values
y_pred = lstm.predict(xpca_test)
y_pred = np.around(y_pred)


In [156]:
t["predicted_score"] =  y_pred

In [157]:
t

,Unnamed: 0,promptId,uniqueId,essay,predicted_score
0,0,1,1_315,Curriculum has been adopted in many schools. T...,3.0
1,1,1,1_214,"I strongly agree with the statement , The tig...",4.0
2,2,1,1_196,Imagination and creativity is the most importa...,3.0
3,3,1,1_178,In our eduction system leaves no room for imag...,3.0
4,4,1,1_201,"I will agree at some what extend, because if w...",3.0
...,...,...,...,...,...
300,300,5,5_146,Earth is a creation of God and everything that...,3.0
301,301,5,5_65,production of arms and weapons in this present...,3.0
302,302,5,5_151,Race to become more powerful can destroy the e...,3.0
303,303,5,5_404,In its attempt to harness the power of the ato...,3.0


In [158]:
t.to_csv("test_prediction.csv",index = False)